In [14]:
import requests
import uuid
import time
headers = {
        "API-KEY": "sk-9cd03c60e0de6b9fad992f905b5e8cbc",
        "Ai-trace-id": str(uuid.uuid4()),   # generate unique ID for each request
        "Content-Type": "application/json"
    }

def generate_video(prompt, duration=5, aspect_ratio="16:9", model="v5",
                   quality="360p", seed=0, negative_prompt="string", watermark=False):
    """
    Call PixVerse API to generate a video from text.
    
    :param prompt: Text prompt for video generation
    :param duration: Duration of the video in seconds
    :param aspect_ratio: Aspect ratio (e.g., "16:9")
    :param model: Model version (e.g., "v5")
    :param quality: Video resolution (e.g., "540p")
    :param seed: Random seed (0 for random)
    :param negative_prompt: Negative prompt to avoid unwanted details
    :param watermark: Whether to add watermark (boolean)
    :return: Response JSON from API
    """

    base_url = "https://app-api.pixverse.ai/openapi/v2/video/text/generate"
    
    
    payload = {
        "aspect_ratio": aspect_ratio,
        "duration": duration,
        "model": model,
        # "motion_mode": "normal",   # optional
        "negative_prompt": negative_prompt,
        # "camera_movement": "zoom_in",   # optional
        "prompt": prompt,
        "quality": quality,
        "seed": seed,
        # "template_id": 302325299692608,   # optional
        "water_mark": watermark
    }
    
    response = requests.post(base_url, headers=headers, json=payload)
    
    try:
        response.raise_for_status()
        return response.json()
    except requests.exceptions.HTTPError as e:
        print(f"HTTP error: {e}")
        print("Response content:", response.text)
        return None

def check_status(video_id):
    VIDEO_URL = f"https://app-api.pixverse.ai/openapi/v2/video/result/{video_id}"
    resp = requests.get(VIDEO_URL, headers=headers)
    resp.raise_for_status()
    data = resp.json()
    if data.get("ErrCode") == 0:
        return data["Resp"]["status"], data["Resp"].get("url")
    else:
        raise RuntimeError(f"Error checking status: {data.get('ErrMsg')}")

# Example usage:
if __name__ == "__main__": 
    result = generate_video(prompt="An avatar doing hand signing asking 'it was nice meeting you' in Auslan")
    video_id = result['Resp']['video_id']
    
    while True:
        status, url = check_status(video_id)
        print(f"Status:{status}")
        if status == 1:
            print(f"Completed! Video URL:{url}")
            break
        time.sleep(5)


Status:5
Status:5
Status:1
Completed! Video URL:https://media.pixverse.ai/pixverse%2Fmp4%2Fmedia%2Fweb%2Fori%2F50123212-3dc7-459e-a5a3-903e761aa5f4_seed207094866.mp4
